<a href="https://colab.research.google.com/github/HayaLababidi/GP_ArabicTextEmotionRecognition/blob/master/GP_char_embedding_(usingembeddinglayer)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation


In [1]:
!pip install gensim
!pip install sklearn
!pip install tensorflow
!pip install keras
!pip install nltk
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.6/dist-packages (1.7.8)


# get required files from drive

In [2]:
#Download all files in gdrive folder
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer 
import string

from googleapiclient.discovery import build
import keras
import random
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Conv1D ,MaxPooling1D , Embedding
from keras.engine.input_layer import Input

import csv


Using TensorFlow backend.


In [0]:


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#charecter embedding files
file_list = drive.ListFile(
    {'q': "'1rdLWzamDZgoZJ7NkPY74zHqa5xTX-5LR' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = f['title']
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  print( f['id'])
  f_.GetContentFile(fname)

f_ = drive.CreateFile({'id': "1HDgAAPkClv-maZ-xmp_avfewuNeFMg0A"})
fname = "Emotional-Tone-Dataset.csv"
f_.GetContentFile(fname)

# Read DataSet

In [0]:

def read_dataset():
    data_df = pd.read_csv("Emotional-Tone-Dataset.csv", encoding="windows-1256")
    X = data_df[['tweet']].values
    Y = data_df[['label']].values
    
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(Y) # need to be global or remembered to use it later
    one_hot_Y=label_binarizer.transform(Y)
    
    return X, one_hot_Y

# convert each character to integer number

In [5]:
def get_dictonary(dataset):
    uniques = ''
    row = ''
    for text in dataset:
        #row = ''
        try:
           row =row + ''.join(set(text[0]))
        except:
            pass
    uniques = uniques.join(set(row))
    length = len(uniques)
    indexes = list(range(length))

    di = dict(zip(uniques, indexes))
    return di , length


def convert_to_int(dataset,dictionary):
    length = 0

    int_dataset = []
    row = []
    for text in dataset:
        length += 1
        row.clear()
        try:
            for char in text[0]:
                number = dictionary[char]
                row.append(number)
        except:
            pass
        int_dataset.append(row)

    return int_dataset

dataset = [['ذَهَبَ مُحَمَّد اِلَى المدرسة صباحا'],
               ['ذَهَبَ مُحَمَّد اِلَى المدرسة ليلا'],
               ['عاد محمد من النادى']]
_dictionary , length = get_dictonary(dataset)
converted_dataset = convert_to_int(dataset, _dictionary)
print(converted_dataset)




[[7, 9, 19, 10, 20, 14, 20, 19, 10, 20, 8, 10, 9, 1, 8, 9, 19, 12], [7, 9, 19, 10, 20, 14, 20, 19, 10, 20, 8, 10, 9, 1, 8, 9, 19, 12], [7, 9, 19, 10, 20, 14, 20, 19, 10, 20, 8, 10, 9, 1, 8, 9, 19, 12]]


# Model with embedding layer

https://keras.io/layers/embeddings/

embedding layer

In [0]:
import keras
import random
from keras.models import Sequential
from keras.callbacks import TensorBoard
import numpy as np

#import read_data

def train_model(length):
    _dropout_rate = 0.2
    _dropout_rate_softmax = 0.5
    _number_of_inputs = 300 #max number of characters per doc(tweet)
    _batch_size = 10
    _kernal_size= 5 #An integer or tuple/list of a single integer
    _pool_size = 3
    _noise_shape = (_batch_size,1,_number_of_inputs)
    _epochs = 100
    _test_size = 0.33 # percentage of test from the dataset
    _Learning_rate = 0.0001
    _input_dim_emb_layer = length # max value of a letter in htis vocab set
    
    _output_dim_emb_layer = 100   #Dimension of the dense embedding.
    _vector_size = _output_dim_emb_layer #vector for each char
    _input_length = 129 # max length in this dataset 

    X_train = np.load('C_X_train.npy')
    X_test = np.load('C_X_test.npy')
    y_train = np.load('C_y_train.npy')
    y_test = np.load('C_y_test.npy')

    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    #print("done input embedding")
    model = Sequential()

    #input
    #model.add(keras.layers.Input(shape=(_number_of_inputs,_vector_size)))

    #Dropout
    #model.add(keras.layers.Dropout(rate=_dropout_rate,input_shape=(_number_of_inputs,_vector_size)))#,noise_shape,random.randint(0,number_of_inputs)))

    #Embedding
    model.add(Embedding(_input_dim_emb_layer, _output_dim_emb_layer,input_length=_input_length))

    #Convolution
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu",input_shape=(X_train.shape[0], X_train.shape[1], _output_dim_emb_layer)))
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

    #Convolution
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

    #Convolution
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

    #Convolution
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))

    #Convolution
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.Conv1D(filters=_vector_size, kernel_size=_kernal_size, strides=1, padding='same', activation="relu"))
    model.add(keras.layers.MaxPooling1D(pool_size = _pool_size, padding='same'))


    #Dropout
    model.add(keras.layers.Dropout(_dropout_rate_softmax))#,noise_shape,random.randint(0,number_of_inputs)))

    #output
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(8, activation="softmax"))

    opt = keras.optimizers.Adam(lr=_Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False);

    model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )

    tensorboard = TensorBoard(log_dir='./log', histogram_freq=1,
                             write_graph=True,
                             write_grads=True,
                             batch_size=_batch_size,
                             write_images=True)

    model.summary()

    # Train the model
    model.fit(
        X_train,
        y_train,
        batch_size=_batch_size,
        epochs=_epochs,
        validation_data=(X_test, y_test),
        shuffle=True,
        callbacks=[tensorboard]
    )
    model.save("trail_1(embedded_layer).h5")

# calling

In [0]:
import numpy as np

_test_size = 0.33 # percentage of test from the dataset 


#read dataset
dataset, one_hot_Y = read_dataset()
_dictionary ,length= get_dictonary(dataset)

#convert each character to integer number
int_dataset = convert_to_int(dataset,_dictionary)
print(np.array(int_dataset).shape)


#split dataset 
C_X_train, C_X_test, C_y_train, C_y_test = train_test_split(int_dataset, one_hot_Y, test_size = _test_size, random_state=42)

#save train_test data
np.save('C_X_train.npy', C_X_train)
np.save('C_X_test.npy', C_X_test)
np.save('C_y_train.npy', C_y_train)
np.save('C_y_test.npy', C_y_test)

#train model
train_model(length)


(10065, 129)
(6743, 129)
(6743, 8)
(3322, 129)
(3322, 8)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 129, 100)          15500     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 129, 100)          50100     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 129, 100)          50100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 43, 100)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 43, 100)           50100     
______________________________

In [0]:
import keras
import random
from keras.models import Sequential
from keras.layers import Embedding
import numpy as np

model = Sequential()
model.add(Embedding(50, 4))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(50, size=(5, 5))
print(input_array)

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape[2])
#assert output_array.shape == (32, 10, 64)